In [1]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import ToTensor
from FaceFeatureDataset import FaceFeatureDataset
import dlib_index as DI
import pandas as pd
import numpy as np

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [2]:
batch_size = 10
epochs = 500

In [3]:
features = pd.read_csv('./outimg/Train/facefeature.csv')
features = features.iloc[0, 1:]
features = np.array(features, dtype=np.float32).reshape(-1,2);
features = np.take(features, DI.LEFT_EYE, axis=0)
print(features)


[[ 0.10154363 -0.1885906 ]
 [ 0.14489932 -0.22053692]
 [ 0.19510067 -0.22281879]
 [ 0.2361745  -0.20456375]
 [ 0.19966443 -0.18174496]
 [ 0.14946309 -0.17946309]]


In [4]:
# 머리      headSize =0, headWidth = 1,
# 이마      foreheadPosition= 2,foreheadSize = 3,
# 턱        jawsPosition =4,jawsSize = 5,
# 아래턱    chinPosition = 6,chinSize = 7,chinPronounced = 8,
# 뺨        lowCheek = 9, cheekPosition = 10, cheekSize = 11,
# 귀        earsPosition = 12, earsRotation =13, earsSize = 14
# 눈        eyeSize = 15, eyePosition= 16, eyeDepth = 17, eyeRotation = 18, eyeDistance = 19, eyeSquint= 20,
# 코        noseSize,nosePosition,noseFlatten,nosePronounced,noseWidth,noseBridge,noseCurve,noseInclination,
# 입        mouthSize,mouthPosition,mouthPronounced,lipsSize

#eyeIndexes = ['eyeSize', 'eyePosition', 'eyeDepth', 'eyeRotation', 'eyeDistance', 'eyeSquint']
featureIndexes = DI.LEFT_EYE
#eyeIndexes = list(range(15, 21))
eyeIndexes = [15]

training_data = FaceFeatureDataset(feature_file="./outimg/Train/facefeature.csv", feature_indexes=featureIndexes,
                                   label_file="./Dataset/Train/csv/train.csv", label_indexes=eyeIndexes)
test_data = FaceFeatureDataset(feature_file="./outimg/Test/facefeature.csv", feature_indexes=featureIndexes,
                               label_file="./Dataset/Test/csv/test.csv", label_indexes=eyeIndexes)

train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# 데이터 로드 확인
for X, y in train_loader:
    # N , Channel, H= width W = height
    print(f"Shape of X [N, F, C]: {X.shape}")
    print(f"Shape of Tensor y: {y.shape} {y.dtype}")
    break

n_total_steps = len(train_loader)

print(X, y)


Shape of X [N, F, C]: torch.Size([10, 6, 2])
Shape of Tensor y: torch.Size([10, 6]) torch.float32
tensor([[[ 0.1025, -0.1928],
         [ 0.1473, -0.2224],
         [ 0.1969, -0.2204],
         [ 0.2376, -0.1959],
         [ 0.1994, -0.1777],
         [ 0.1476, -0.1796]],

        [[ 0.0966, -0.1891],
         [ 0.1428, -0.2245],
         [ 0.1972, -0.2272],
         [ 0.2407, -0.2054],
         [ 0.2026, -0.1755],
         [ 0.1455, -0.1755]],

        [[ 0.0978, -0.1902],
         [ 0.1444, -0.2234],
         [ 0.1960, -0.2256],
         [ 0.2395, -0.1995],
         [ 0.2008, -0.1792],
         [ 0.1466, -0.1770]],

        [[ 0.0986, -0.1896],
         [ 0.1428, -0.2249],
         [ 0.1949, -0.2245],
         [ 0.2385, -0.1994],
         [ 0.2000, -0.1751],
         [ 0.1451, -0.1755]],

        [[ 0.0955, -0.1905],
         [ 0.1412, -0.2270],
         [ 0.1997, -0.2288],
         [ 0.2444, -0.2015],
         [ 0.2041, -0.1729],
         [ 0.1457, -0.1738]],

        [[ 0.0984, -0.

In [5]:
num_classes = len(eyeIndexes)
print(num_classes)

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(12, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, num_classes),
            #nn.ReLU(),
        )
    

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


6
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=12, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=6, bias=True)
  )
)


In [6]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)        
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print('batch',  batch)
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
# Training
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, criterion, optimizer)    
print("Done!")

Epoch 1
-------------------------------
loss: 0.299201  [    0/  100]
Epoch 2
-------------------------------
loss: 0.296234  [    0/  100]
Epoch 3
-------------------------------
loss: 0.244715  [    0/  100]
Epoch 4
-------------------------------
loss: 0.261374  [    0/  100]
Epoch 5
-------------------------------
loss: 0.223051  [    0/  100]
Epoch 6
-------------------------------
loss: 0.190582  [    0/  100]
Epoch 7
-------------------------------
loss: 0.184771  [    0/  100]
Epoch 8
-------------------------------
loss: 0.163282  [    0/  100]
Epoch 9
-------------------------------
loss: 0.152263  [    0/  100]
Epoch 10
-------------------------------
loss: 0.114833  [    0/  100]
Epoch 11
-------------------------------
loss: 0.116991  [    0/  100]
Epoch 12
-------------------------------
loss: 0.108413  [    0/  100]
Epoch 13
-------------------------------
loss: 0.085754  [    0/  100]
Epoch 14
-------------------------------
loss: 0.063937  [    0/  100]
Epoch 15
------

In [9]:
PATH = './face_eye.pth'
torch.save(model.state_dict(), PATH)

In [10]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    print('test size', size )
    # num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)            
            print('pred =', pred)
            #print('loss', loss)
            #print('real', y)



In [11]:

test(test_loader, model, criterion)

print("Done !")

test size 100
pred = tensor([[0.4881, 0.5094, 0.5319, 0.4936, 0.5220, 0.4966],
        [0.4874, 0.5092, 0.5315, 0.4931, 0.5216, 0.4963],
        [0.4883, 0.5099, 0.5324, 0.4940, 0.5224, 0.4968],
        [0.4884, 0.5097, 0.5321, 0.4937, 0.5220, 0.4966],
        [0.4884, 0.5096, 0.5320, 0.4937, 0.5219, 0.4967],
        [0.4872, 0.5094, 0.5316, 0.4933, 0.5220, 0.4964],
        [0.4883, 0.5097, 0.5322, 0.4938, 0.5222, 0.4965],
        [0.4874, 0.5093, 0.5316, 0.4934, 0.5220, 0.4964],
        [0.4884, 0.5095, 0.5318, 0.4938, 0.5221, 0.4964],
        [0.4872, 0.5095, 0.5317, 0.4933, 0.5222, 0.4965]], device='cuda:0')
pred = tensor([[0.4886, 0.5092, 0.5315, 0.4936, 0.5215, 0.4962],
        [0.4875, 0.5093, 0.5316, 0.4933, 0.5217, 0.4966],
        [0.4881, 0.5095, 0.5319, 0.4935, 0.5218, 0.4965],
        [0.4886, 0.5099, 0.5324, 0.4940, 0.5223, 0.4967],
        [0.4882, 0.5095, 0.5318, 0.4937, 0.5221, 0.4965],
        [0.4878, 0.5092, 0.5315, 0.4934, 0.5218, 0.4964],
        [0.4882, 0.5092, 0